### **Introduction to Arachne**
Arachne is a Python package for graph analysis that is built as an extension to Arkouda, a Python package for analysis on tabular data, akin to NumPy and Pandas. In this notebook we will show examples on how to run each algorithm that has been implemented on different types of graphs: undirected, directed, and property.

In [41]:
import arkouda as ak
import arachne as ar
import scipy as sp
import networkx as nx
import matplotlib.pyplot as plt
import os

In [ ]:
# NOTE: Make sure to change the server name to whatever is applicable in your environment. If running locally, then use only ak.connect().
ak.connect("n32", 5555)

### **Graph Generation and Loading**
Graphs can be built from existing data or generated with our suite of random graph generators. The preferred way to load a graph into memory is from Arkouda arrays, however we provide a method to read a graph in from matrix market format or randomly generate some graphs.

In [5]:
# Read in a graph from a matrix market file.
absolute_path_to_karate = os.path.abspath("data/karate.mtx")
karate = ar.read_matrix_market_file(absolute_path_to_karate)

In [ ]:
# Generate a random graph using any variety of random generators available.
n = 100
rmat_graph = ar.rmat(15, create_using=ar.Graph)
gnp_graph = ar.gnp(n, 0.75, create_using=ar.Graph)
rtree_graph = ar.random_tree(n, create_using=ar.Graph)
ws_graph = ar.watts_strogatz_graph(n, 25, 0.56, create_using=ar.Graph)
graph_list = [rmat_graph, gnp_graph, rtree_graph, ws_graph]
for g in graph_list:
    print(f"Generated graph has {len(g)} vertices and {g.size()} edges")

In [7]:
# Create a property graph from Arkouda dataframes, usually read in from HDF5, Parquet, or CSV files. For demonstrative purposes, we create some random dataset here.
n = 1_000           # Number of vertices.
m = 1_000_000       # Number of edges.
k = 2               # Value to cap the randomness at.

In [8]:
# Create variously different random arrays of different types: integers, unsigned integers, floats, booleans, strings, and categoricals.
src_array = ak.randint(0, n, m, dtype=ak.dtype('int64'), seed=2)
dst_array = ak.randint(0, n, m, dtype=ak.dtype('int64'), seed=4)
int_array = ak.randint(-1, k, m, dtype=ak.dtype('int64'), seed=6)
uint_array = ak.randint(0, k, m, dtype=ak.dtype('uint64'), seed=8)
real_array = ak.randint(0, k, m, dtype=ak.dtype('float64'), seed=10)
bool_array = ak.randint(0, k, m, dtype=ak.dtype('bool'), seed=12)
strings_array = ak.random_strings_uniform(0, k, m, characters="abcdefghijklmonpqrstuvwxyz", seed=14)
categorical_array = ak.Categorical(ak.random_strings_uniform(0, k, m, characters="abcdefghijklmonpqrstuvwxyz", seed=14))

In [9]:
# Initialize an empty graph object.
prop_graph = ar.PropGraph()

In [10]:
# Create a dataframe with the edge data.
test_edge_dict = {
    "src":src_array,
    "dst":dst_array,
    "data1":int_array,
    "data2":uint_array,
    "data3":real_array,
    "data4":bool_array,
    "data5":strings_array,
    "data6":categorical_array
}
test_edge_df = ak.DataFrame(test_edge_dict)

In [ ]:
test_edge_df

In [12]:
# Load in the edge attributes, with sorts the edges and handles storing their data.
prop_graph.load_edge_attributes(test_edge_df, source_column="src", destination_column="dst", relationship_columns=["data5", "data1"])

In [13]:
# Create sizes for vertex information.
m = len(prop_graph)
k = 2

In [14]:
# Create data of different types for vertices.
int_array = ak.randint(-1, k, m, dtype=ak.dtype('int64'), seed=6)
uint_array = ak.randint(0, k, m, dtype=ak.dtype('uint64'), seed=8)
real_array = ak.randint(0, k, m, dtype=ak.dtype('float64'), seed=10)
bool_array = ak.randint(0, k, m, dtype=ak.dtype('bool'), seed=12)
strings_array = ak.random_strings_uniform(0, k, m, characters="abcdefghijklmonpqrstuvwxyz", seed=14)
categorical_array = ak.Categorical(ak.random_strings_uniform(0, k, m, characters="abcdefghijklmonpqrstuvwxyz", seed=14))

In [15]:
# Create a dataframe with vertex data.
test_node_dict = {
    "nodes":prop_graph.nodes(),
    "data1":int_array,
    "data2":uint_array,
    "data3":real_array,
    "data4":bool_array,
    "data5":strings_array,
    "data6":categorical_array
}
test_node_df = ak.DataFrame(test_node_dict)

In [ ]:
test_node_df

In [17]:
# Load in the vertex data.
prop_graph.load_node_attributes(test_node_df, node_column="nodes", label_columns=["data5", "data2"])

### **Graph Processing and Querying**
Treating the graphs as dataframes allows us to exploit Arkouda's array searches to generate subgraphs in seconds.

In [18]:
# Create filters for vertices.
def node_filter(node_attributes):
    return node_attributes["data2"] == 0

In [19]:
# Create filters for edges.
def edge_filter(edge_attributes):
    return edge_attributes["data1"] > -1

In [ ]:
# Create different subgraphs from each demo and together.
subgraph_nodes = prop_graph.subgraph_view(filter_node=node_filter)
print(f"Subgraph generated with edge size: {subgraph_nodes.size()}")
subgraph_edges = prop_graph.subgraph_view(filter_edge=edge_filter)
print(f"Subgraph generated with edge size: {subgraph_edges.size()}")
subgraph_together = prop_graph.subgraph_view(filter_node=node_filter, filter_edge=edge_filter)
print(f"Subgraph generated with edge size: {subgraph_together.size()}")

### **Graph Algorithms**
Let's now revisit the karate graph from above and do some analyses with Arachne and NetworkX together! First, let's start out by reading the matrix market file again, both with Arachne and NetworkX.

In [21]:
# First, let's build the graph with Arachne.
filepath = os.path.abspath("data/karate.mtx")
G = ar.read_matrix_market_file(filepath)

edge_src, edge_dst = G.edges()
edge_src = edge_src.to_list()
edge_dst = edge_dst.to_list()
edge_list = []
for (u,v) in zip(edge_src,edge_dst):
    edge_list.append((u,v))
nodes = G.nodes()

In [ ]:
# Secondly, let's build the graph with NetworkX. NOTE: Arachne has a direct read_matrix_market_file method whereas NetworkX requires you to use SciPy to read in the matrix market file.
fh = open(filepath, "rb")
H = nx.from_scipy_sparse_array(sp.io.mmread(fh))
print(f"Let's make sure that both graphs have the same number of vertices and edges. Arachne graph has {len(G)} vertices and {G.size()} edges. NetworkX has {len(H)} vertices and {H.size()} edges.")

We are also able to display our graph with NetworkX methods by exporting our edgelist and building a graph from the edges stored by Arachne. **Note: Visualization is resource-intensive, so you can really only do it for small graphs. However, we perform these steps to show how Arachne can complement NetworkX and how graphs can be exchanged between them.**

In [ ]:
nx_display = nx.Graph()
nx_display.add_edges_from(edge_list)

pos = nx.kamada_kawai_layout(nx_display)
nx.draw_networkx(nx_display, pos, with_labels=True)
plt.show()

Below we also see how we can extract a degree view for a graph to see the dispersion of degrees amongst the vertices. This also works for directed graphs by using the `G.in_degree()` and `G.out_degree()` methods.

In [ ]:
degrees = G.degree()
print(degrees)

Using Arkouda arrays we can also extract the node with maximum degree.

In [ ]:
print(f"Node {G.nodes()[ak.argmax(degrees)]} has maximum degree of {ak.max(degrees)}")

### **Breadth-First Search**

Given a graph $G$ (as defined above) and a source vertex $s$, breadth-first search (BFS) traverses the graph in a level-centric manner. It can return the tree inherently generated during BFS, the vertices found at each layer, or an ordering of nodes as predecessors or successors. For our implementation we opted to mimic NetworkX's `bfs_layers` function that returns the nodes and the layer they belong to.

Our BFS method, written in Chapel, expands the next frontier in parallel on each locale, if it is run on a distributed cluster. Each locale iterates over the frontier, and if there are any local edges, it inspects the neighbors, and if not yet visited, updates the `depth` (layer) pdarray. The final `depth` array is returned which can be used to provide a different view of a graph.

In [ ]:
d = ar.bfs_layers(G, 1)
print(d)

We may also extract a histogram of the sizes of each level computed by BFS. This can be a useful manner in seeing how how big each group of vertices at a particular level is.

In [ ]:
d_histogram = ak.histogram(d, bins=ak.max(d)+1)
print(d_histogram[0])

From the above we can see that at level 0 we obviously only have one vertex. Then, we can see that at level 1 we have 16, level 2 we have 9, and level 3 we have 8. The maximum level is 3. What happens if we change the source vertex? Let's see the results of bfs_layers for each vertex.

In [ ]:
for i in range(1,len(G)+1):
    d = ar.bfs_layers(G, i)
    d_histogram = ak.histogram(d, bins=ak.max(d)+1)
    print(f"Size of breadth-first search layers for vertex {i}: {d_histogram[0]}")

From the output above we notice some main things:
1. Starting BFS from different sources arise to different number of layers for the output of each vertex.
2. Some nodes have the same number of vertices at each layer. 
3. Vertex 19 provides the largest number of layers for any of the vertices. 

Let's color the nodes for node 19 and show the output.

In [ ]:
d = ar.bfs_layers(G, 19)
color_dict = {0:"blue", 1:"green", 2:"red", 3:"purple", 4:"orange", 5:"yellow"}
print(d)

color_map = []
nodes = nx_display.nodes()
for u in nodes:
    color_map.append(color_dict[d[u-1]])
    
print(nx_display.nodes)

pos = nx.kamada_kawai_layout(nx_display)
nx.draw_networkx(nx_display, pos, with_labels=True, node_color=color_map)
plt.show()

print(f"degrees = {degrees}")

In the returned image we can see the first frontier are the green vertices $<0, 1, 33>$. If we look at the degree view of the graph we notice that 0, 1, and 33 are in the top 4 of vertices with most degree, which now makes sense why vertex 19 had the largest expanded frontier. 

### **Connected Components**

Given a graph G, (as defined above) if there is a path from a vertex $u$ to every other vertex $v$ then the graph is said to be connected. If there is not a path, then the graph is said to be disconnected and composed of multiple connected components. There may be a large number of connected components of varying sizes in a graph.

One of the most ancient manners of calculating connected components involve running BFS until every vertex has been visited, at every iteration looking for the node whose value `-1` and start BFS from it. These steps are repeated until all the vertices have been labeled. The other is using union-find to build a tree induced by each connected component. For our method we use a lbel propagation technique that sends the minimum vertex label to all the other vertices in a connected component.

To run our connected components, you just have to call the `connected_components()` method. **We use one of the randomly generated graphs above for this example**.

In [ ]:
cc = ar.connected_components(rmat_graph)
print(cc)

We can use Arkouda methods to get the size of each component in our file. This will let us know which vertices will be included in the induced subgraph of the largest component.

In [ ]:
g = ak.GroupBy(cc)
keys, count = g.size()
label_of_largest_component = ak.argmax(count)
label_of_smallest_component = ak.argmin(count)
print(f"The largest component is labeled: {keys[label_of_largest_component]}")
print(f"The smallest component is labeled: {keys[label_of_smallest_component]}")

### **Triangle Counting**
One of the most basic community structures that can be found in graphs are triangles. Triangles are 3-cliques in an undirected graph which means three nodes are strongly connected to each other. Triangle counting can be used to detect how cohesive communities are, the more triangles there are, the better connected a community is. It can also be used to drive other graph analytical algorithms such as centrality measures (triangle centrality) and substructure detection (k-truss analytics).

Our triangle counting method involves inspecting every edge in a graph and the adjacency lists of both endpoints to find the intersection point. Most algorithms perform list intersection, for our case we perform a binary search of every vertex in the smaller adjacency list into the bigger one. This allows us to efficiently find triangles with less work.

Please note that you have to divide the total number of triangles found by 3.

In [ ]:
G_tri = ar.triangles(G)
print(f"The whole graph has {G_tri/3} triangles.")

You can also pass an array of vertex names to return only the number of triangles those vertices belong to.

In [ ]:
G_tris = ar.triangles(G, ak.array([1,19]))
print(f"Vertex 1 has {G_tris[0]} triangles and vertex 19 has {G_tris[1]} triangles.")

### **Truss Analytics**

Given a graph $G$ (as defined above), the $k$-truss of a graph is one where every edge is a part of least $k-2$ triangles. This creates a cohesive subgraph where only the edges that meet these requirement are kept. It involves recalculating the number of triangles multiple times, which we avoid by tracking the support of each edge at every iteration of the algorithm. We provide novel algorithmic implementations for three truss analytical algorithms:
1. $k$-truss
2. max-truss
3. truss decomposition

Below we run `k_truss` and display the edges that belong to at least 2 triangles. We could build an induced subgraph from these edges in the same manner as we showed above.

In [ ]:
kt = ar.k_truss(G, 4)
print(kt)

Here, we see that the maximum truss of this graph is 5. This means that the $k$-truss is no longer defined for any value of $k$ larger than 5.

In [ ]:
mt = ar.max_truss(G)
print(mt)

Lastly, the truss decomposition shows the trussness of every edge. This means that the largest value of k for every edge is the one presented in the edge index. 

In [ ]:
td = ar.truss_decomposition(G)
print(td)

Using similar coloring steps as above, we can actually color the edges of the graph. The steps for that are below.

In [ ]:
color_dict = {5:"red", 4:"purple", 3:"green", 2:"blue"}
edge_color = []

for i in range(G.size()):
    edge_color.append(color_dict[td[i]])
    
pos = nx.spring_layout(nx_display, seed=200)
nx.draw_networkx(nx_display, pos, with_labels=True, edge_color=edge_color)
plt.show()

### **Triangle Centrality**

Triangle centrality is a centrality measure to decide how important a vertex is based off how many triangles surround a particular vertex. It is calculated based off the following formula as seen in the paper titled "Triangle Centrality" by Paul Burkhardt.

$$TC(v) = \frac{\frac{1}{3} \sum_{u \in N_{\Delta}^{+}(v)}{\Delta(u)} + \sum_{w \in (N(v) \backslash N_{\Delta}(v))} {\Delta (w)}}{\Delta(G)}$$

Where $N(v)$ is the neighborhood set of a vertex $v$, $N_{\Delta}(v)$ is the set of neighbors that are in triangles with $v$, and $N_{\Delta}^{+}$ is the closed set that includes $v$. There is not an equivalent metric of this form available in NetworkX.

In [ ]:
tce = ar.triangle_centrality(G)
print(f"One of the most important vertices in the graph is: {G.nodes()[ak.argmax(tce)]}")

### **Subgraph Isomorphism**

The problem of motif finding in graphs is one of pattern matching with a smaller subgraph to search inside of a larger host graph. Here, we have an implementation of parallel VF2 that returns the subgraph mappings of vertices inside of the graph that contain the same structure as the given subgraph. Currently, our subgraph isomorphism method works only for property graphs. Further, it returns monomorphisms.

In [39]:
G_directed = ar.PropGraph()
edges_to_add = G.edges()
G_directed.add_edges_from(edges_to_add[0], edges_to_add[1])

In [40]:
subgraph = ar.PropGraph()
src = [0, 1, 2, 1]
dst = [1, 2, 0, 3]
subgraph.add_edges_from(ak.array(src), ak.array(dst))

In [41]:
isos = ar.subgraph_isomorphism(G_directed, subgraph)

In [ ]:
print(f"We found {len(isos)/4} triangles with tails inside of the karate graph")

### **Diameter**

The diameter of a graph is the longest of the shortest paths between two vertices in a graph. Here, we should an experimental version that approximates the diameter with connected components.

In [ ]:
di = ar.diameter(G)
print(f"The diameter of karate is {di}") 

### **Well-Connected Components**

Performs steps to confirm if a component within a community is well-connected or not.

In [ ]:
import pandas as pd

cluster_dict = {}

# Read the file and populate the dictionary
with open('/scratch/users/md724/DataSets/wcc/test_clustering.tsv', 'r') as file:
    for line in file:
        # Remove any leading/trailing whitespace
        line = line.strip()
        # Skip empty lines
        if not line:
            continue
        # Split the line into node and cluster number
        node_str, cluster_str = line.split('\t')
        node = int(node_str)
        cluster_num = int(cluster_str)
        # Add the node to the corresponding cluster
        if cluster_num in cluster_dict:
            cluster_dict[cluster_num].append(node)
        else:
            cluster_dict[cluster_num] = [node]

# Find the cluster with the maximum number of nodes
max_cluster_num = None
max_cluster_nodes = []
max_cluster_size = 0

for cluster_num, nodes in cluster_dict.items():
    cluster_size = len(nodes)
    if cluster_size > max_cluster_size:
        max_cluster_size = cluster_size
        max_cluster_num = cluster_num
        max_cluster_nodes = nodes

# Assign the nodes of the biggest cluster to the variable 'cluster'
cluster = max_cluster_nodes

# Print the cluster and its size
#print(f"cluster = {cluster};")
print(f"Cluster {max_cluster_num} is the biggest cluster with {max_cluster_size} nodes")


# Read the TSV file using pandas
network_df = pd.read_csv("/scratch/users/md724/DataSets/wcc/test_network.tsv", sep="\t", header=None, names=["src", "dst"])
network_df['type'] = 'T1'

# Print the pandas DataFrame (optional, just to verify)
print(network_df)
print(network_df.head())



# %%
# Concatenate 'src' and 'dst' columns and find unique elements for nodes
unique_nodes = pd.concat([network_df['src'], network_df['dst']]).unique()

# Number of unique nodes
num_unique_nodes = len(unique_nodes)
#num_unique_nodes

# %%
# Number of edges is the length of the DataFrame
num_edges = len(network_df)

# Calculate graph density
d = num_edges / (num_unique_nodes * (num_unique_nodes - 1))
print(f"Graph Density: {d}")

print(f"Number of unique nodes: {num_unique_nodes}")
print(f"Number of edges: {num_edges}")


# %%

# Convert the pandas DataFrame to an Arkouda DataFrame
ak_network_df = ak.DataFrame(network_df)
ak_network_df

# %%
# Create an Arachne PropGraph and load the edge attributes
ar_network_graph = ar.PropGraph()
ar_network_graph.load_edge_attributes(ak_network_df, source_column="src", destination_column="dst", relationship_columns=["type"])


# %%
# Get all nodes and create node attributes
all_nodes = ak.concatenate([ak_network_df['src'], ak_network_df['dst']])
unique_nodes = ak.unique(all_nodes)
lbls = ak.array(["1"] * unique_nodes.size)
network_node_df = ak.DataFrame({"nodes": unique_nodes, "lbls": lbls})
network_node_df


# %%

ar_network_graph.load_node_attributes(network_node_df, node_column="nodes", label_columns=["lbls"])
ar_network_graph

# %%
print("Running Arachne")
filePath = "/scratch/users/md724/DataSets/wcc/clustering.tsv"
print(type(ar_network_graph))  # Check if it's PropGraph
print(type(filePath)) 
clusters = ar.well_connected_components(ar_network_graph,filePath)

print("clusters = ", clusters)